# 02. DataCleaning

### `train.csv`を更新する
- typoの更新（kiler_whale -> killer_whale）
- typoの更新（bottlenose_dolpin -> bottlenose_dolphin）
- whale, dolphinのclass追加
- imageのhight, weightclassの追加
- (できそうであれば)RGBのmean, max, minの数字を追加
- (できそうであれば)簡易的にsegmentationした場合の背景のBGR値とobjectのBGR値

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import os
import cv2

### 1. typoの更新(kiler_whale, killer_whale)

In [2]:
base_path = 'res/'
# base_path = '/kaggle/input/happy-whale-and-dolphin/'

train_df = pd.read_csv(base_path+'train.csv')
print(train_df['species'].unique())

['melon_headed_whale' 'humpback_whale' 'false_killer_whale'
 'bottlenose_dolphin' 'beluga' 'minke_whale' 'fin_whale' 'blue_whale'
 'gray_whale' 'southern_right_whale' 'common_dolphin' 'kiler_whale'
 'pilot_whale' 'dusky_dolphin' 'killer_whale' 'long_finned_pilot_whale'
 'sei_whale' 'spinner_dolphin' 'bottlenose_dolpin' 'cuviers_beaked_whale'
 'spotted_dolphin' 'globis' 'brydes_whale' 'commersons_dolphin'
 'white_sided_dolphin' 'short_finned_pilot_whale' 'rough_toothed_dolphin'
 'pantropic_spotted_dolphin' 'pygmy_killer_whale' 'frasiers_dolphin']


In [3]:
train_df['species'] = train_df['species'].str.replace('bottlenose_dolpin','bottlenose_dolphin')
train_df['species'] = train_df['species'].str.replace('kiler_whale','killer_whale')
train_df["species"].replace({"beluga": "beluga_whale", "globis": "globis_whale"}, inplace=True)

In [4]:
train_df['species'].unique()

array(['melon_headed_whale', 'humpback_whale', 'false_killer_whale',
       'bottlenose_dolphin', 'beluga_whale', 'minke_whale', 'fin_whale',
       'blue_whale', 'gray_whale', 'southern_right_whale',
       'common_dolphin', 'killer_whale', 'pilot_whale', 'dusky_dolphin',
       'long_finned_pilot_whale', 'sei_whale', 'spinner_dolphin',
       'cuviers_beaked_whale', 'spotted_dolphin', 'globis_whale',
       'brydes_whale', 'commersons_dolphin', 'white_sided_dolphin',
       'short_finned_pilot_whale', 'rough_toothed_dolphin',
       'pantropic_spotted_dolphin', 'pygmy_killer_whale',
       'frasiers_dolphin'], dtype=object)

In [5]:
# creating a new column 
train_df['label'] = train_df.species.map(lambda x: 'dolphin' if 'dolphin' in x else 'whale')

In [6]:
train_df.head()

,image,species,individual_id,label
0,00021adfb725ed.jpg,melon_headed_whale,cadddb1636b9,whale
1,000562241d384d.jpg,humpback_whale,1a71fbb72250,whale
2,0007c33415ce37.jpg,false_killer_whale,60008f293a2b,whale
3,0007d9bca26a99.jpg,bottlenose_dolphin,4b00fe572063,dolphin
4,00087baf5cef7a.jpg,humpback_whale,8e5253662392,whale


In [7]:
test_df = pd.read_csv('sample_submission.csv')
test_df.head()

,image,predictions
0,000110707af0ba.jpg,37c7aba965a5 114207cab555 a6e325d8e924 19fbb96...
1,0006287ec424cb.jpg,37c7aba965a5 114207cab555 a6e325d8e924 19fbb96...
2,000809ecb2ccad.jpg,37c7aba965a5 114207cab555 a6e325d8e924 19fbb96...
3,00098d1376dab2.jpg,37c7aba965a5 114207cab555 a6e325d8e924 19fbb96...
4,000b8d89c738bd.jpg,37c7aba965a5 114207cab555 a6e325d8e924 19fbb96...


### height, widthを追加する

In [8]:
from tqdm import tqdm as tqdm
import imagesize

def get_imgsize_train(row):
    
    image_path = os.path.join('res/train_images', row['image'])    
    row['width'], row['height'] = imagesize.get(image_path) 
    # row['width'], row['height'], _ = image.shape
    image = np.array(cv2.imread(image_path))
    for p in [0, 5, 10, 25, 50, 75, 90, 100]:
    # for p in [10, 25, 50, 75]:
        row['p_{}_B'.format(p)], row['p_{}_G'.format(p)], row['p_{}_R'.format(p)] = np.percentile(image, p, axis=(0, 1))

    return row

def get_imgsize_test(row):
    
    image_path = os.path.join('res/test_images', row['image'])    
    row['width'], row['height'] = imagesize.get(image_path) 
    image = cv2.imread(image_path)
    image = np.array(image)
    for p in [0, 5, 10, 25, 50, 75, 90, 100]:
    # for p in [10, 25, 50, 75]:
        row['p_{}_B'.format(p)], row['p_{}_G'.format(p)], row['p_{}_R'.format(p)] = np.percentile(image, p, axis=(0, 1))

    return row

In [9]:
# Train
tqdm.pandas(desc='Train ')
train_df = train_df.progress_apply(get_imgsize_train, axis=1)
train_df.to_csv('updated_train.csv', index=False)
# Test
tqdm.pandas(desc='Test ')
test_df = test_df.progress_apply(get_imgsize_test, axis=1)
test_df.to_csv('updated_test.csv', index=False)# test_df = test_df.progress_apply(get_imgsize, axis=1)


Test : 100%|██████████| 27956/27956 [10:08:52<00:00,  1.31s/it]  


In [10]:
# train_df.to_csv('updated_train.csv', index=False)
# test_df.to_csv('updated_test.csv', index=False)